In [20]:
import PyPDF2
import re
import numpy as np
from ollama import embed

with open("./UNOClassicManualGerman.pdf", "rb") as file:
    reader = PyPDF2.PdfReader(file)
    raw_text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])

paragraphs = [p.strip() for p in re.split(r"\n{2,}|(?<=\.)\s*\n", raw_text) if len(p.strip()) > 20]

response = embed(model='nomic-embed-text', input=paragraphs)
embeddings = np.array(response['embeddings'])

# Frage des Users
user_question = "Wie kann ich den Spieler, der nach mir kommt, aussetzen lassen?"

query_embedding = np.array(embed(model='nomic-embed-text', input=user_question)['embeddings'])

normalized_embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
normalized_query = query_embedding / np.linalg.norm(query_embedding)
cosine_similarities = np.dot(normalized_embeddings, normalized_query.T).flatten()
sorted_indices = np.argsort(-cosine_similarities)

top_k = 3
retrieved_paragraphs = [paragraphs[i] for i in sorted_indices[:top_k]]
retrieved_text = "\n\n".join(retrieved_paragraphs)

print(retrieved_text)

prompt = f"""
# Question

{user_question}

# Identity

You are a game assistant for the UNO game and answer questions from the user about rules of game.

# Instructions

1. Only answer based on the context of the manual
2. If there is no answer or not a definite answer to the Question in the manual, say it so
3. Find out in which language the Question was asked
4. At the end translate your answer to the language of the Question

# Manual
{retrieved_text}
"""

from llama_index.llms.ollama import Ollama
from llama_index.core.llms import ChatMessage

llm = Ollama(model="mistral:instruct", model_provider="ollama", request_timeout=300)

messages = [
    ChatMessage(role="user", content=prompt)
]

chat_response = llm.chat(messages)
print(chat_response)

Spieler, die anderen Spielern Vorschläge machen, welche Karten sie legen sollen, müssen 2 Karten von 
dem KARTENSTOCK aufnehmen.

AUSSETZEN KARTE 
Nachdem diese Karte gelegt wurde, wird der nächste Spieler "übersprungen". Die 
Karte kann nur auf eine Karte mit entsprechender Farbe oder eine andere Aussetzen 
Karte gelegt werden. Wenn diese Karte zu Beginn des Spiels gezogen wird, wird der 
Spieler zur Linken des Gebers "übersprungen" und der Spieler zur Linken dieses 
Spielers setzt das Spiel fort.

Sobald ein Spieler keine Karten mehr hat, ist das Ablegen beendet. Die Punkte werden 
zusammengezählt (siehe PUNKTE) und das Spiel beginnt von neuem.
assistant:  In the context of the UNO game manual, to skip the next player, you can play the 'Skip' card. However, it can only be placed on a card with the same color or another Skip card. If this card is drawn at the beginning of the game, the player to the left of the giver is skipped and play continues to the left of that player. Once a pla